In [63]:
import numpy as np
import math

# System parameters
M = 1.0  # kg
m = 0.1  # kg
g = -9.8  # m/s^2
l = 0.5  # m
mu_c = 0.0005
mu_p = 0.000002
delta_t = 0.3
# as center points were to infer the state, 0.02 was too small to
# enable a state transition, so I altered the stepsize of delta_t

# model parameters
num_states = 162
num_actions = 2
actions = [-10, 10]

# Define the bounds for each parameter
bounds = [
    [-12, -6, -1, 0, 1, 6, 12], # theta
    [-2.4, -0.8, 0.8, 2.4], # x
    [-100, -50, 50, 100], # theta_dot
    [-5, -0.5, 0.5, 5] # x_dot
]

In [32]:
# calculate the values of center point for each parameter
values = []
for b in bounds:
    mean = np.convolve(b, [0.5, 0.5], mode='valid')
    values.append(mean)
# print(values)

[array([-9. , -3.5, -0.5,  0.5,  3.5,  9. ]), array([-1.6,  0. ,  1.6]), array([-75.,   0.,  75.]), array([-2.75,  0.  ,  2.75])]


In [33]:
# calculate center point
centers = np.array(np.meshgrid(*values)).T.reshape(-1, 4)
# print(centers)
# print(centers.shape)

In [34]:
# calculate the index of given state
def find_indices(value, bounds):
    # value = np.array(value)
    indices = []
    for i, dim_bounds in enumerate(bounds):
        if value[i] < dim_bounds[0] or value[i] > dim_bounds[-1]:
            return False  # if any value out of bound: False
        lower_bound_idx = np.searchsorted(dim_bounds, value[i], side='right') - 1
        indices.append(lower_bound_idx)
    if value[0] == 12:
        indices[0] = indices[0] - 1
    if value[1] == 2.4:
        indices[1] = indices[1] - 1
    return indices

In [35]:
# test
# o = find_indices([12, 2.4, -100, -5], bounds)  # find indices in 'values'
# print(o)

[5, 2, 0, 0]


In [50]:
# dynamic formula
def theta_aaa(M, m, g, l, mu_c, mu_p, theta, theta_dot, x, x_dot, action):
    # theta, theta_dot, x, x_dot = S[0], S[1], S[2], S[3]
    F = action
    rt = math.radians(theta)
    sintheta = math.sin(rt)
    costheta = math.cos(rt)
    # print('radias',rt)
    # print(sintheta)
    theta_a = (g * sintheta + (costheta*(-F-m*l*pow(theta_dot,2)*sintheta + mu_c*np.sign(x_dot))/(M+m)) - (mu_p * theta_dot / (m*l)))/(l*(4.0/3.0-(m*pow(costheta,2))/(m+M)))
    # print('theta_a:', theta_a)
    return theta_a

def x_aaa(M, m, g, l, mu_c, mu_p, theta, theta_dot, x, x_dot, action, theta_a):
    # theta, theta_dot, x, x_dot = S[0], S[1], S[2], S[3]
    F = action
    rt = math.radians(theta)
    sintheta = math.sin(rt)
    costheta = math.cos(rt)
    # print('p', pow(theta_dot,2))
    x_a = (F + m*l*(pow(theta_dot,2) * sintheta - theta_a * costheta) - mu_c*np.sign(x_dot)) / (M+m)
    # print('x_a:', x_a)
    return x_a

def calculate_next_state(theta, x, theta_dot, x_dot, action):  # exact value of next_state
    # t = math.radians(theta)
    # p = x
    # t_d = math.radians(theta_dot)
    # p_d = x_dot
    # print('t, t_d', t, t_d)
    
    theta_a = theta_aaa(M, m, g, l, mu_c, mu_p, theta, theta_dot, x, x_dot, action)
    x_a = x_aaa(M, m, g, l, mu_c, mu_p, theta, theta_dot, x, x_dot, action, theta_a)
    
    x_dot += delta_t * x_a
    x += delta_t * x_dot
    theta_dot += delta_t * theta_a
    theta += delta_t * theta_dot
    return theta, x, theta_dot, x_dot

In [53]:
p1, p2, p3, p4 = calculate_next_state(-0.5, 0, 0.0, 0.0, 10)  # theta, x, theta_dot, x_dot
print('theta, x, theta_dot, x_dot', p1, p2, p3, p4)

theta, x, theta_dot, x_dot -1.8046257734096907 0.8774807316935379 -4.348752578032302 2.9249357723117932


In [38]:
def next_state(S, action):  # index of next_state
    S = np.array(S)
    # print('S:', S)
    theta, x, theta_dot, x_dot = S[0], S[1], S[2], S[3]
    next_theta, next_x, next_theta_dot, next_x_dot = calculate_next_state(theta, theta_dot, x, x_dot, action)
    value = [next_theta, next_x, next_theta_dot, next_x_dot]
    # print('Value:', value)
    result = find_indices(value, bounds)  # new state index
    if not result:
        return [9, 9, 9, 9]  # terminal state: failure
    result_state = [values[i][result[i]] for i in range(len(result))]
    # print('result_state:', result_state)
    return result_state  # non-failure state: return center point of the box to infer that state

In [39]:
# test
# t = next_state([-0.5, 0, 0.0, 0.0], 10)
# print(t)  # theta, x, theta_dot, x_dot

[9, 9, 9, 9]


In [54]:
# policy initialization
# policy = np.array([[[[[0, 1] for _ in range(3)] for _ in range(3)] for _ in range(3)] for _ in range(6)])
policy = np.ones((6, 3, 3, 3, num_actions)) / num_actions
print(policy)
# V initialization
V = np.zeros((6, 3, 3, 3))

# reward
def calculate_reward(state):
    if np.all(state == [9, 9, 9, 9]):
        return 0
    else:
        return 1

[[[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]]



 [[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]]



 [[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]


In [55]:
# Policy Iteration
def policy_iteration(policyy, gamma=0.9, theta=1e-6):
    # flag = 0
    while True:
        # Policy Evaluation
        # ite = 0
        while True:  # (for i in range(100)?)
            delta = 0
            for s in range(num_states):
                state = np.array(centers[s])  # current state, centers: 162*1
                # print('Current State:', state)
                # print(type(state))
                if np.all(state == np.array([9, 9, 9, 9], dtype=np.float32)):
                    continue  # terminal state

                # calculate current v
                state_index = find_indices(state, bounds)  # index for current state
                s1, s2, s3, s4 = state_index[0], state_index[1], state_index[2], state_index[3]
                v_old = V[s1][s2][s3][s4]  # current v

                # calculate a
                i = np.argmax(policyy[s1][s2][s3][s4])
                a = actions[i]

                # calculate new state
                new_state = next_state(state, a)  # new state represents by center

                index_in_grid = find_indices(new_state, bounds)  # find index in V array
                if not index_in_grid:
                    continue  # terminal state
                a, b, c, d = index_in_grid[0], index_in_grid[1], index_in_grid[2], index_in_grid[3]  # new state index

                # reward of this move
                reward = calculate_reward(new_state)

                # update V
                V[s1][s2][s3][s4] = reward + gamma * V[a][b][c][d]
                v = V[s1][s2][s3][s4]

                # calculate delta
                delta = max(delta, abs(v - v_old))
                print('delta:', delta)

            # ite += 1

            if delta < theta:  #  or ite == 100
                # print('ite:', ite)
                # if ite < 2:
                #     flag += 1
                break

        # Policy Improvement
        policy_stable = True
        for s in range(num_states):
            state = np.array(centers[s])  # current state, centers:162*1
            find_index = find_indices(state, bounds)  # find index in V array
            m, n, p, q = find_index[0], find_index[1], find_index[2], find_index[3]
            # print(m, n, p, q)
            old_action = np.argmax(policyy[m][n][p][q])  # index

            action_values = np.zeros(num_actions)

            for i in range(num_actions):
                a = actions[i]
                new_state = next_state(state, a)  # new state represents by center
                reward = calculate_reward(new_state)
                index_in_grid = find_indices(new_state, bounds)  # find index in V array
                if not index_in_grid:
                    continue  # terminal state

                a, b, c, d = index_in_grid[0], index_in_grid[1], index_in_grid[2], index_in_grid[3]
                # print(a, b, c, d)
                action_values[i] = reward + gamma * V[a][b][c][d]

            best_action = np.argmax(action_values)  # index of best

            if old_action != best_action:
                policy_stable = False
            policyy[m][n][p][q] = np.eye(num_actions)[best_action]  # only the best has a prob of 1

        print('policy_stable', policy_stable)
        if policy_stable:  #  or flag == 3
            break

    return policyy

# Policy Iteration
print('previous policy:', policy)
optimal_policy = policy_iteration(policy)

# Print Policy and V
print("Optimal Policy:")
print(optimal_policy)
print("V:")
print(V)

previous policy: [[[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]]



 [[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]]



 [[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]

delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.00017868760652639537
delta: 0.0

In [56]:
ssstate = [-0.5, 0, 0.0, 0.0]

for i in range(1000):
    ind = find_indices(ssstate, bounds)
    a1, a2, a3, a4 = ind[0], ind[1], ind[2], ind[3]
    policy_ssstate = optimal_policy[a1][a2][a3][a4]
    actionnn = actions[np.argmax(policy_ssstate)]
    next = next_state(ssstate, actionnn)
    print('action:', policy_ssstate)
    print('state after action:', next)
    ssstate = next

action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
action: [1. 0.]
state after action: [-3.5, 0.0, 0.0, 0.0]
action: [0. 1.]
state after action: [-3.5, 1.6, 0.0, 2.75]
actio

In [59]:
print('Value for negative theta_dot:', V[:,:,0,0], V[:,:,0,1], V[:,:,0,2])
print('Value for theta_dot = 0:', V[:,:,1,0], V[:,:,1,1], V[:,:,1,2])
print('Value for positive theta_dot:', V[:,:,2,0], V[:,:,2,1], V[:,:,2,2])

Value for negative theta_dot: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Value for theta_dot = 0: [[9.99999607 9.99999607 9.99999607]
 [9.99999607 9.99999607 9.99999607]
 [9.99999607 9.99999607 9.99999607]
 [9.99999607 9.99999607 9.99999607]
 [9.99999607 9.99999607 9.99999607]
 [9.99999607 9.99999607 9.99999607]] [[9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]] [[9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]]
Value for positive theta_dot: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0

In [60]:
print('Policy for negative theta_dot:', policy[:,:,0,0], policy[:,:,0,1], policy[:,:,0,2])
print('Policy for theta_dot = 0:', policy[:,:,1,0], policy[:,:,1,1], policy[:,:,1,2])
print('Policy for positive theta_dot:', policy[:,:,2,0], policy[:,:,2,1], policy[:,:,2,2])

Policy for negative theta_dot: [[[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]] [[[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]] [[[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]]]
Policy for theta_dot = 0: [[[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]] [[[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0.

In [61]:
# Value Iteration

def value_iteration(policyyy, gamma=0.9, theta=1e-6):
    while True:
        delta = 0
        for s in range(num_states):
            state = np.array(centers[s])  # current state, centers: 162*1
            if np.all(state == np.array([9, 9, 9, 9], dtype=np.float32)):
                continue  # terminal state
                
            # calculate current v
            state_index = find_indices(state, bounds)  # index for current state
            s1, s2, s3, s4 = state_index[0], state_index[1], state_index[2], state_index[3]
            v_old = V[s1][s2][s3][s4]  # current v
            
            action_values = np.zeros(num_actions)
            
            for i in range(2):
                myaction = actions[i]
                new_state = next_state(state, myaction)  # new state represents by center
                reward = calculate_reward(new_state)
                index_in_grid = find_indices(new_state, bounds)  # find index in V array
                if not index_in_grid:
                    continue  # terminal state
                a, b, c, d = index_in_grid[0], index_in_grid[1], index_in_grid[2], index_in_grid[3]
                action_values[i] = reward + gamma * V[a][b][c][d]
            
            best_action = np.argmax(action_values)
            
            V[s1][s2][s3][s4] = action_values[best_action]
            policyyy[s1][s2][s3][s4] = np.eye(2)[best_action]
            v = V[s1][s2][s3][s4]  # update v
            
            delta = max(delta, abs(v - v_old))
            
        if delta < theta:
            break
    return policyyy

In [62]:
# policy initialization
# policy = np.array([[[[[0, 1] for _ in range(3)] for _ in range(3)] for _ in range(3)] for _ in range(6)])
policy = np.ones((6, 3, 3, 3, num_actions)) / num_actions
# print(policy)

# V initialization
V = np.zeros((6, 3, 3, 3))

# reward function
def calculate_reward(state):
    if np.all(state == [9, 9, 9, 9]):
        return 0
    else:
        return 1

# Value Iteration
print('previous policy:', policy)
optimal_policy = value_iteration(policy)

# Print Policy and V
print("Optimal Policy:")
print(optimal_policy)
print("V:")
print(V)

[[[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]]



 [[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]]



 [[[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]]


  [[[0.5 0.5]
    [0.5 0.5]
    [0.5 0.5]]

   [[0.5 0.5]


In [64]:
print('Value for negative theta_dot:', V[:,:,0,0], V[:,:,0,1], V[:,:,0,2])
print('Value for theta_dot = 0:', V[:,:,1,0], V[:,:,1,1], V[:,:,1,2])
print('Value for positive theta_dot:', V[:,:,2,0], V[:,:,2,1], V[:,:,2,2])

print('Policy for negative theta_dot:', policy[:,:,0,0], policy[:,:,0,1], policy[:,:,0,2])
print('Policy for theta_dot = 0:', policy[:,:,1,0], policy[:,:,1,1], policy[:,:,1,2])
print('Policy for positive theta_dot:', policy[:,:,2,0], policy[:,:,2,1], policy[:,:,2,2])

Value for negative theta_dot: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Value for theta_dot = 0: [[9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]
 [9.99999647 9.99999647 9.99999647]] [[9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]
 [9.99999682 9.99999682 9.99999682]] [[9.99999714 9.99999714 9.99999714]
 [9.99999714 9.99999714 9.99999714]
 [9.99999714 9.99999714 9.99999714]
 [9.99999714 9.99999714 9.99999714]
 [9.99999714 9.99999714 9.99999714]
 [9.99999714 9.99999714 9.99999714]]
Value for positive theta_dot: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0